In [2]:
import cv2
from ultralytics import YOLOWorld
import supervision as sv
import os

In [3]:
# Путь к изображениям
PATH_TO_IMAGES = "sample_substraction"
# Размер батча
BATCH_SIZE = 5
# Порог для NMS
NMS_THRESHOLD = 1e-9
# Порог уверенности распознавания класса
CONFIDENCE = 0.45
# Размер изображения
image_size = (640, 640)
# Список картинок
list_of_images = os.listdir(PATH_TO_IMAGES)
list_of_images_with_relative_path = [os.path.join(PATH_TO_IMAGES, image) for image in list_of_images]
# Задаем промпт для модели
license_plate_description = """
russian car license plate: 
- white rectangular metal plate
- black cyrillic characters
- format: 1 letter, 3 digits, 2 letters, regional code
- example: A123BC777
- not a car manufacturer badge or model emblem
- not a part of vehicle body (door, bumper, headlight, etc.)
"""

In [4]:
# Инициализируем модель
model = YOLOWorld("yolov8x-worldv2.pt")

# Запихиваем промпт в список классов
classes = [license_plate_description, ""]
model.set_classes(classes)

# Создаем датасет
dataset = sv.DetectionDataset(classes=classes, 
                              images=list_of_images_with_relative_path, 
                              annotations={list_of_images_with_relative_path[i]: None 
                                           for i in range(len(list_of_images_with_relative_path))})

# Итерация по списку картинок: обрабатываем картинки батчами
for i in range(0, len(list_of_images_with_relative_path), BATCH_SIZE):
    batch = list_of_images_with_relative_path[i:BATCH_SIZE+i+1]
    lst_of_cv2_instances = [cv2.imread(image) for image in batch]

    # Запускаем детекцию (подаем батч картинок) с порогом уверенности 
    # TODO если классов несколько - для каждого из них должен быть разный порог уверенности
    results = model.predict(batch, conf=CONFIDENCE, imgsz=image_size)

    # Итерация по каждой детекции в батче
    for index in range(len(results)):
        # Получение детекций
        detections = sv.Detections.from_ultralytics(results[index])

        # Считаем кол-во пикселей на изображении
        image_area = image_size[0] * image_size[1]

        #  Non-Maximum Suppression (NMS) постобработка для удаления дублирующихся прямоугольников
        filtered_detections = detections.with_nms(threshold = NMS_THRESHOLD)

        # Инициализация экземпляров классов для боксов и подписей
        annotator = sv.BoxAnnotator()
        label_annotator = sv.LabelAnnotator()

        # вывод доп информации (уверенность)
        labels = [
            f"{confidence:.2f}"
            for confidence
            in filtered_detections.confidence
        ]

        # Непосредственно, рисуем боксы и подписи
        image_with_bounding_box = annotator.annotate(scene = lst_of_cv2_instances[index], 
                                            detections = filtered_detections)
        annotated_image = label_annotator.annotate(scene = image_with_bounding_box, 
                                                detections = filtered_detections, 
                                                labels=labels)
        
        # Сохраняем размеченную картинку
        cv2.imwrite(f"annotated_{batch[index]}", annotated_image)

        # Вносим в датасет инфу по детекциям для конкретной картинки
        dataset.annotations[batch[index]] = filtered_detections



0: 640x640 1 
russian car license plate: 
- white rectangular metal plate
- black cyrillic characters
- format: 1 letter, 3 digits, 2 letters, regional code
- example: A123BC777
- not a car manufacturer badge or model emblem
- not a part of vehicle body (door, bumper, headlight, etc.)
, 49.8ms
1: 640x640 1 
russian car license plate: 
- white rectangular metal plate
- black cyrillic characters
- format: 1 letter, 3 digits, 2 letters, regional code
- example: A123BC777
- not a car manufacturer badge or model emblem
- not a part of vehicle body (door, bumper, headlight, etc.)
, 49.8ms
2: 640x640 3 
russian car license plate: 
- white rectangular metal plate
- black cyrillic characters
- format: 1 letter, 3 digits, 2 letters, regional code
- example: A123BC777
- not a car manufacturer badge or model emblem
- not a part of vehicle body (door, bumper, headlight, etc.)
s, 49.8ms
3: 640x640 1 
russian car license plate: 
- white rectangular metal plate
- black cyrillic characters
- format: 1

In [13]:
# Разбиваем датасет на train, test и val
train_dataset, test_dataset = dataset.split(split_ratio=0.8, random_state=42, shuffle=True)
# Дополнительно разобьем test, чтобы получить val
test_dataset, val_dataset = test_dataset.split(split_ratio=0.5, random_state=42)

In [18]:
dct_of_datasets = {'train': train_dataset,
                   'test': test_dataset,
                   'valid': val_dataset}

In [19]:
for dataset_type, dataset_instance in dct_of_datasets.items():
    # Экспортируем в YOLO-формат (data_yaml генерируется невалидный для CVAT, нужно создавать самому)
    dataset_instance.as_yolo(
    images_directory_path=f"license_plate_detection_dataset/{dataset_type}/images",
    annotations_directory_path=f"license_plate_detection_dataset/{dataset_type}/labels",
    )